In [1]:
%reload_ext autoreload
%autoreload 2
import data_02_conflictSolvers as solvers
import data_01_conflictModel as model

#import data_04_spSolvers as solvers
import data_03_gmcrUtilities as gmcrUtils
import gmcr
import numpy as np
from utilities import *

# Non cooperative stability

- Step 0: Load data
- Step 1: Identify which baseline policy one wants to use
- Step 2: Create a game using GMCR-py package and load conflict actors
- Step 3: Identify preference ranking between all the policies for all the actors
- Step 4: Solve the game to find the equilibrium conditions
- Step 5: Store the records

In [2]:
payoffs = load_data("../model/data/processed/50yr_best.pickle")

In [3]:
payoffs

,hydropower_revenue,atomic_power_plant_discharge,baltimore_discharge,chester_discharge,recreation,environment,flood_risk
policy1,20.452941,0.662558,0.449381,0.580373,0.123929,-0.263545,-1.344648
policy2,27.709023,0.413537,0.574970,0.331784,0.160357,-0.152379,-1.247479
policy3,28.001663,0.276144,0.372764,0.214182,0.106429,-0.114024,-1.237239
policy4,24.098135,0.652890,0.498963,0.576507,0.123214,-0.248537,-1.443722
policy5,28.001663,0.276144,0.372764,0.214182,0.106429,-0.114024,-1.237239
policy6,24.256598,0.504577,0.572013,0.421109,0.106429,-0.203868,-1.447469


In [4]:
baseline = {'hydropower_revenue':39.30, 'atomic_power_plant_discharge':0.633,
       'baltimore_discharge':1.00, 'chester_discharge':1.00, 'recreation':0.00, 'environment':0.1058,
       'flood_risk':1.633}

In [5]:
baseline = {'hydropower_revenue':0.0, 'atomic_power_plant_discharge':0.0,
       'baltimore_discharge':0.00, 'chester_discharge':0.00, 'recreation':0.00, 'environment':-1,
       'flood_risk':-2}

In [6]:
decisions = ['policy1', 'policy2', 'policy3']
payoffs = payoffs[payoffs.index.isin(decisions)]

In [7]:
payoffs = payoffs.append(baseline, ignore_index = True)

In [8]:
# pick out the actors of interest
payoffs = payoffs[['atomic_power_plant_discharge', 'hydropower_revenue']]

In [9]:
myConflict = gmcr.model.ConflictModel()

#### Add players to the conflict arena

In [10]:
hydropower_revenue = gmcr.model.DecisionMaker(myConflict, "hydropower_revenue")
myConflict.decisionMakers.append(hydropower_revenue)
atomic_power_plant_discharge = gmcr.model.DecisionMaker(myConflict, "atomic_power_plant_discharge")
myConflict.decisionMakers.append(atomic_power_plant_discharge)
#baltimore_discharge = gmcr.model.DecisionMaker(myConflict, "baltimore_discharge")
# myConflict.decisionMakers.append(baltimore_discharge)
# chester_discharge = gmcr.model.DecisionMaker(myConflict, "chester_discharge")
# myConflict.decisionMakers.append(chester_discharge)
# recreation = gmcr.model.DecisionMaker(myConflict, "recreation")
# myConflict.decisionMakers.append(recreation)
# environment = gmcr.model.DecisionMaker(myConflict, "environment")
# myConflict.decisionMakers.append(environment)
# flood_risk = gmcr.model.DecisionMaker(myConflict, "flood_risk")
# myConflict.decisionMakers.append(flood_risk)
# print(myConflict.decisionMakers)

In [11]:
myConflict.useManualPreferenceRanking = True

In [12]:
# Add all the decisions the players must chose from except the baseline 
nr_decisions = payoffs.shape[0]-1
nr_players = payoffs.shape[1]
for i in range(0, nr_players):
    for j in decisions:
        myConflict.decisionMakers[i].addOption(j)

In [13]:
%%time
myConflict.recalculateFeasibleStates()

2 players
3 decisions
[(9, '100100'), (10, '010100'), (12, '001100'), (17, '100010'), (18, '010010'), (20, '001010'), (33, '100001'), (34, '010001'), (36, '001001')] temp
Wall time: 999 µs


### Step 3: Identify preference ranking

In [14]:
# Create a temporary list mapping each state to the policy decision that it would result in. Each state is a combination of strategies an actor chooses
temp = [(x,y) for x,y in zip([feas for feas in myConflict.feasibles.name], [feas for feas in myConflict.feasibles])]

In [15]:
# create a dictionary mapping every policy to all the states where it gets picked
dict = {}
for i in set([feas for feas in myConflict.feasibles.name]):
    dict[i] = []
for i in temp:
    dict[i[0]].append(i[1])

In [16]:
# create ordinal ranking between all the options
# rename the baseline decision
ranks = payoffs.rank(ascending = False)
ranks = ranks.reset_index()
ranks['index']= ranks['index'].replace(nr_decisions, 'baseline')

In [17]:
actors = [dm.name for dm in myConflict.decisionMakers]

In [18]:
payoffs

,atomic_power_plant_discharge,hydropower_revenue
0,0.662558,20.452941
1,0.413537,27.709023
2,0.276144,28.001663
3,0.000000,0.000000


In [19]:
ranks

,index,atomic_power_plant_discharge,hydropower_revenue
0,0,1.0,3.0
1,1,2.0,2.0
2,2,3.0,1.0
3,baseline,4.0,4.0


In [20]:
# create a preference ranking for each actor based on their ranking
# Add that to the conflict
for i in range((nr_players)):
    rank_order = list(ranks.sort_values(by = actors[i])['index'])
    index_map = {v:j for j, v in enumerate(rank_order)}
    sorted_list = sorted(dict.items(), key=lambda x: index_map[x[0]])
    myConflict.decisionMakers[i].preferenceRanking = list(list(zip(*sorted_list))[1])

In [21]:
# calculate preferences for every actor based on the ranking
for i in range((nr_players)):
     myConflict.decisionMakers[i].calculatePreferences()

In [22]:
hydropower_revenue.payoffs

array([7, 6, 6, 6, 8, 6, 6, 6, 9])

In [23]:
%reload_ext autoreload
%autoreload 1
import data_02_conflictSolvers as solvers

In [24]:
%%time
lSolver = solvers.LogicalSolver(myConflict)

[8, 16, 32] other COs moves
[1, 2, 4] focal COs moves
from state 0
hydropower_revenue for dm
[] reachable
from state 32
hydropower_revenue for dm
[36, 34, 33] reachable
8
7
6
from state 16
hydropower_revenue for dm
[20, 18, 17] reachable
5
4
3
from state 48
hydropower_revenue for dm
[] reachable
from state 8
hydropower_revenue for dm
[12, 10, 9] reachable
2
1
0
from state 40
hydropower_revenue for dm
[] reachable
from state 24
hydropower_revenue for dm
[] reachable
from state 56
hydropower_revenue for dm
[] reachable
[1, 2, 4] other COs moves
[8, 16, 32] focal COs moves
from state 0
atomic_power_plant_discharge for dm
[] reachable
from state 4
atomic_power_plant_discharge for dm
[36, 20, 12] reachable
8
5
2
from state 2
atomic_power_plant_discharge for dm
[34, 18, 10] reachable
7
4
1
from state 6
atomic_power_plant_discharge for dm
[] reachable
from state 1
atomic_power_plant_discharge for dm
[33, 17, 9] reachable
6
3
0
from state 5
atomic_power_plant_discharge for dm
[] reachable
from

In [132]:
myConflict.feasibles.toOrdered

{5: 1, 6: 2, 9: 3, 10: 4}

In [134]:
lSolver.conflict.feasibles.toOrdered[10]-1

3

In [136]:
lSolver.conflict.feasibles.toOrdered[9]-1

2

In [126]:
hydropower_revenue.reachability

array([[0, 1, 0, 0],
       [1, 0, 0, 0],
       [0, 0, 0, 1],
       [0, 0, 1, 0]])

In [33]:
%%time
lSolver.findEquilibria()

Wall time: 1.23 ms


In [35]:
decisions = np.array(myConflict.feasibles.name)

In [35]:
np.where(np.array(myConflict.feasibles.name) != 'baseline')

(array([0, 4, 8], dtype=int64),)

In [90]:
#hydropower_revenue.preferenceRanking

In [36]:
print(f'Nash Equilibrium: {set(decisions[lSolver.nashEquilibria])}',
f'GMR Equilirbium: {set(decisions[lSolver.gmrEquilibria])}',
f'SEQ Equilibrium: {set(decisions[lSolver.seqEquilibria])}',
f'SIM Equilibrium: {set(decisions[lSolver.simEquilibria])}',
f'SEQ SIM Equilibrium: {set(decisions[lSolver.seqSimEquilibria])}',
f'SMR Equilirbium: {set(decisions[lSolver.smrEquilibria])}')

Nash Equilibrium: {'0', '2', '1'} GMR Equilirbium: {'0', '2', 'baseline', '1'} SEQ Equilibrium: {'0', '2', '1'} SIM Equilibrium: {'0', '2', 'baseline', '1'} SEQ SIM Equilibrium: {'0', '2', 'baseline', '1'} SMR Equilirbium: {'0', '2', '1'}


In [222]:
lSolver.gmrEquilibria

array([False,  True,  True,  True, False,  True,  True,  True, False])

In [149]:
hydropower_revenue.reachability[2:]* hydropower_revenue.payoffMatrix[2:]

array([[ 0,  0,  0,  2],
       [ 0,  0, -2,  0]])

In [144]:
hydropower_revenue.preferenceRanking

[[3], [0], [1, 2]]

In [145]:
hydropower_revenue.payoffMatrix[2:]

array([[ 1,  0,  0,  2],
       [-1, -2, -2,  0]])

In [148]:
np.nonzero(hydropower_revenue.reachability[2, :] *
                           hydropower_revenue.payoffMatrix[2 :] > 0)

(array([0], dtype=int64), array([3], dtype=int64))

In [102]:
lSolver.UIs(hydropower_revenue,2)

[0]

In [223]:
lSolver.gmr(hydropower_revenue,0)

(False,
 'From state 1 (decimal 9, payoff 6), hydropower_revenue has UIs available to: \n   state 2 (decimal 10, payoff 9),\n   state 3 (decimal 12, payoff 9)\nCheck for sanctioning...\n\nstate 1 (decimal 9, payoff 6) is unstable by GMR for focal DM hydropower_revenue, since other DMs cannot effectively sanction a move to state 2 (decimal 10, payoff 9)')